In [3]:
from datetime import datetime
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time




,date,open,high,low,close,ohlc4,volume,SMA
0,2022-06-05 21:00:00,29923.0,29960.0,29792.0,29937.0,29903.00,9.600126e+06,NaN
1,2022-06-05 22:00:00,29937.0,30055.0,29847.0,30000.0,29959.75,1.933518e+07,NaN
2,2022-06-05 23:00:00,30000.0,30025.0,29867.0,29897.0,29947.25,1.128383e+07,NaN
3,2022-06-06 00:00:00,29897.0,30376.0,29865.0,30280.0,30104.50,2.298991e+07,NaN
4,2022-06-06 01:00:00,30280.0,30905.0,30280.0,30825.0,30572.50,4.407975e+07,NaN
...,...,...,...,...,...,...,...,...
1495,2022-08-07 04:00:00,22993.0,23029.0,22944.0,22951.0,22979.25,5.817889e+06,23132.708333
1496,2022-08-07 05:00:00,22951.0,22999.0,22950.0,22971.0,22967.75,3.709893e+06,23128.640625
1497,2022-08-07 06:00:00,22971.0,23024.0,22970.0,23007.0,22993.00,2.834178e+06,23122.729167
1498,2022-08-07 07:00:00,23007.0,23032.0,22998.0,23017.0,23013.50,2.364440e+06,23118.187500


In [11]:
file = "df_2022_hr_capped.csv"

df_2022_hr = pd.read_csv(file)
df_2022_hr

,date,open,high,low,close,ohlc4,volume,SMA,25x_liq_long,25x_liq_short
0,2022-06-05 21:00:00,29923.0,29960.0,29792.0,29937.0,29903.00,9.600126e+06,NaN,NaN,NaN
1,2022-06-05 22:00:00,29937.0,30055.0,29847.0,30000.0,29959.75,1.933518e+07,NaN,NaN,NaN
2,2022-06-05 23:00:00,30000.0,30025.0,29867.0,29897.0,29947.25,1.128383e+07,NaN,NaN,NaN
3,2022-06-06 00:00:00,29897.0,30376.0,29865.0,30280.0,30104.50,2.298991e+07,NaN,NaN,NaN
4,2022-06-06 01:00:00,30280.0,30905.0,30280.0,30825.0,30572.50,4.407975e+07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1495,2022-08-07 04:00:00,22993.0,23029.0,22944.0,22951.0,22979.25,5.817889e+06,23132.708333,22350.442834,22136.562998
1496,2022-08-07 05:00:00,22951.0,22999.0,22950.0,22971.0,22967.75,3.709893e+06,23128.640625,22346.512681,22132.670455
1497,2022-08-07 06:00:00,22971.0,23024.0,22970.0,23007.0,22993.00,2.834178e+06,23122.729167,22340.801127,22127.013557
1498,2022-08-07 07:00:00,23007.0,23032.0,22998.0,23017.0,23013.50,2.364440e+06,23118.187500,22336.413043,22122.667464


In [10]:
# create a graph object
fig = go.Figure()

#set up layout
fig.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
fig.add_trace(
    go.Candlestick(
        x=df_2022_hr['date'],
        open=df_2022_hr['open'],
        high=df_2022_hr['high'],
        low=df_2022_hr['low'],
        close=df_2022_hr['close'],
        name="BTCUSD"
    )
)


fig.add_trace(
    go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['SMA'], name="SMA(48)")
)


fig.show()

In [15]:
#sample visual for liquidation levels: need EMA does not work with SMA
Fig2 = go.Figure()

#set up layout
Fig2.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
Fig2.add_trace(
    go.Candlestick(
        x=df_2022_hr['date'],
        open=df_2022_hr['open'],
        high=df_2022_hr['high'],
        low=df_2022_hr['low'],
        close=df_2022_hr['close'],
        name="BTCUSD"
    )
)


Fig2.add_trace(
    go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['25x_liq_long'], name="25x Rolling Liqs(long)"),
)

Fig2.add_trace(
  go.Scatter(x=df_2022_hr['date'], y=df_2022_hr['25x_liq_short'], name="25x Rolling Liqs(short)")
)

Fig2.show()
